[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dina-lab3D/tutorials/blob/main/NanoNet/net.ipynb)

#NanoNet 

NanoNet is a novel deep learning-based end-to-end modeling tool that given a sequence directly produces the 3D coordinates of the Cβ atoms of the entire VH domain. It can be used in order to predict structures of nanobodies, VH regions of mAbs and VB regions of TCRs.

**NanoNet architecture**:



<br>
<img src=https://drive.google.com/uc?id=1DdACpv5loaOnKbrIIlJSygUUmt9dRUut width="2000">






In [ ]:
#@title Download modeller, first get license key from **[here](https://salilab.org/modeller/registration.html)** .
from IPython.display import clear_output
license_key = '' #@param {type:"string"}
#MODELIRANJE
!wget https://salilab.org/modeller/10.1/modeller-10.1.tar.gz
!tar -zxf modeller-10.1.tar.gz
!echo "MODELLER extraction completed"
%cd modeller-10.1
#And we prepare a file containing the minimal setup elements
#For installing, including a license key
with open('modeller_config', 'a') as f:
  f.write("3\n")
  f.write("/content/compiled/MODELLER\n")
#ADD YOUR LICENSE KEY HERE!
  f.write(f"{license_key}\n")
!./Install < modeller_config
!echo "MODELLER set up completed"

%cd /content/
#Creating a symbolic link
%cd modeller-10.1
!ln -sf /content/compiled/MODELLER/bin/mod10.1 /usr/bin/
%cd /content/
#Checking if MODELLER works
!mod10.1 | awk 'NR==1{if($1=="usage:") print "MODELLER succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

with open("/content/compiled/MODELLER/modlib/modeller/config.py", "r") as file:
  lines = file.readlines()

with open("/content/compiled/MODELLER/modlib/modeller/config.py", "w") as file:
  file.write(lines[0])
  file.write(f"license = '{license_key}'\n")
clear_output()

### Importing packages and get the relevent files

In [ ]:

!pip install Bio
!pip install import-ipynb
!pip install py3Dmol

!wget https://raw.githubusercontent.com/dina-lab3D/tutorials/main/NanoNet/6zrv.pdb 
!wget https://raw.githubusercontent.com/dina-lab3D/tutorials/main/NanoNet/modeller_side_chains.py 
!wget https://github.com/dina-lab3D/tutorials/raw/main/NanoNet/train_input.npy.zip 
!wget https://github.com/dina-lab3D/tutorials/raw/main/NanoNet/train_labels.npy.zip
!wget https://raw.githubusercontent.com/dina-lab3D/tutorials/main/NanoNet/utils.ipynb 
!wget https://raw.githubusercontent.com/dina-lab3D/tutorials/main/NanoNet/align.py

!git clone https://github.com/dina-lab3D/NanoNet --quiet

!unzip train_input.npy.zip
!unzip train_labels.npy.zip

clear_output()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import py3Dmol
import import_ipynb
import utils

clear_output()

### Set the hyper parameters for the convolutional neural network

In [ ]:
###############################################################################
#                                                                             #
#                        Network Hyper-Parameters                             #
#                                                                             #
###############################################################################

#@markdown ---
#@markdown ### number of ResNet blocks for the first ResNet and the kernel size.

RESNET_1_BLOCKS = 3 #@param [1, 2, 3,4, 5] {type:"raw"}
RESNET_1_KERNEL_SIZE = 15 #@param [7, 15, 25] {type:"raw"}
RESNET_1_KERNEL_NUM = 64 #@param [32, 64, 128] {type:"raw"}

#@markdown ---
#@markdown ### number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 1  #@param [1, 3, 5] {type:"raw"}
RESNET_2_KERNEL_SIZE = 5 #@param [3, 5, 7] {type:"raw"}
RESNET_2_KERNEL_NUM = 140 #@param [70, 140] {type:"raw"}
DILATION = [1,2,4,8,16] 

#@markdown ---
#@markdown ### percentage of dropout for the dropout layer
DROPOUT = 0.25 #@param [0.0, 0.1, 0.25, 0.5] {type:"raw"}


#@markdown ---
#@markdown ### number of epochs, Learning rate and Batch size
EPOCHS = 100 #@param [50, 100, 125] {type:"raw"}
LR = 0.001 #@param [0.0001, 0.001, 0.01] {type:"raw"}
BATCH = 16 #@param [16, 32, 64, 128] {type:"raw"}

### Define a ResNet layer

In [ ]:
def resnet_1(input_layer): 
    """
    ResNet layer - input -> BatchNormalization -> Relu -> Conv1D -> BatchNormalization -> Relu -> Conv1D -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for i in range(RESNET_1_BLOCKS):

        batch_layer = layers.BatchNormalization()(input_layer)
        conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(batch_layer)
        relu_layer = layers.Activation('relu')(conv1d_layer)

        batch_layer = layers.BatchNormalization()(relu_layer)
        conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(batch_layer)
        relu_layer = layers.Activation('relu')(conv1d_layer)

        input_layer = layers.Add()([relu_layer, input_layer])

    return input_layer

### Define a Dilated ResNet layer


In [ ]:
def resnet_2(input_layer):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> Relu -> dilated Conv1D -> BatchNormalization -> Relu -> dilated Conv1D -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for i in range(RESNET_2_BLOCKS):
        for d in DILATION:

            batch_layer = layers.BatchNormalization()(input_layer)
            conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', dilation_rate=d)(batch_layer)
            relu_layer = layers.Activation('relu')(conv1d_layer)

            batch_layer = layers.BatchNormalization()(relu_layer)
            conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', dilation_rate=d)(batch_layer)
            relu_layer = layers.Activation('relu')(conv1d_layer)

            input_layer = layers.Add()([relu_layer, input_layer])


    return input_layer

### Build the entire neural network architecture 

In [ ]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: Keras Model
    """
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer)

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer)

    # dropout -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    dropout_layer = layers.Dropout(DROPOUT)(resnet_layer)

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM // 2, RESNET_2_KERNEL_SIZE, padding="same", activation="elu")(dropout_layer)

    # Dense -> shape = (NB_MAX_LENGTH, OUTPUT_SIZE)
    # outpur_layer = layers.Conv1D(utils.OUTPUT_SIZE, RESNET_2_KERNEL_SIZE, padding="same")(conv1d_layer)

    outpur_layer = layers.Dense(utils.OUTPUT_SIZE)(conv1d_layer)

    return tf.keras.Model(input_layer, outpur_layer, name="my_network")


### Fucntion for ploting the training and validation losses

In [ ]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'], label='Training loss')
    axes.plot(history.history['val_loss'], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("model_loss_history") 


### Load the training dataset and split it to training and validation sets


X - the input for the network, Nb sequence (as one-hot representation):

original sequence - VGG...Q
one-hot representation:

<br>
<img src=https://drive.google.com/uc?id=1jWfEkFvs6JSNdpvaH8ZEbdDnOD1xCSTn width="300">


Y - the output of the network, backbone + CB coordinates of the solved structure (after alignment to a reference nanobody)


<br>
<img src=https://drive.google.com/uc?id=1ui5n-o6xxCV13zd6tKFEXPW2eradH95M width="300"> <img src=https://drive.google.com/uc?id=1SdiqFwJEgvUAMPc8GV51dI25_A_iSxJP width="400"> 


In [ ]:

model = build_network()


# you can load here your input and output data

# X = numpy array of shape (2141,NB_MAX_LENGTH,FEATURE_NUM) of all the data input.
# Y = numpy array of shape (2141,NB_MAX_LENGTH,OUTPUT_SIZE) of all the data labels.

X = np.load("train_input.npy")
Y = np.load("train_labels.npy")

# split into validation and test sets as you like
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1,  shuffle=True)


### Define the loss function - Mean squared error (equivalent to RMSD) + CA distance loss

<br>
<img src=https://drive.google.com/uc?id=1YogRx01bTicTak8jzFDgzNxMPCnUdPVC width="380">

<br>
<img src=https://drive.google.com/uc?id=1cfdTQ2nlvewzcehxp49d1aOz_q9ZU7Ge width="350">




In [ ]:
def ca_distance_loss(real, pred):
    """
    masked flat-bottom l1 loss for antibody ca distances.
    """

    mse_loss = tf.math.reduce_mean((real- pred)**2, axis=-1)
    mse_loss = tf.math.reduce_mean(mse_loss, axis=-1)

    mask = tf.cast(tf.math.logical_not(tf.math.reduce_all(tf.math.equal(real, 0), axis=-1)), dtype=real.dtype)
    pred_ca = pred[:, :, 3:6]
    rolled_pred_ca = tf.roll(pred_ca, shift=1, axis=-2)
    ca_dist = tf.math.reduce_sum((pred_ca - rolled_pred_ca) ** 2, axis=-1)
    ca_dist = (tf.math.sqrt(ca_dist) - 3.77) ** 2
    ca_mask = tf.math.multiply(mask, tf.roll(mask, shift=1, axis=-1))
    ca_dist *= ca_mask
    ca_dist = (tf.reduce_sum(ca_dist, axis=-1) / tf.reduce_sum(ca_mask,axis=-1))

    return mse_loss + ca_dist 

### Set loss function + algorithm for optimization

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer = optimizer, loss=["mse", ca_distance_loss])
save_callback = tf.keras.callbacks.ModelCheckpoint("my_model",save_best_only=True, verbose=1)

### Train the neural network

In [ ]:
# fit model (use EPOCH for epoch parameter and BATCH for batch_size parameter)
net_history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=EPOCHS, verbose=1, batch_size=BATCH, callbacks=[save_callback])
plot_val_train_loss(net_history)

### Predict the structure of a new Nb (6zrv) using the trained neural network

In [ ]:
# model = tf.keras.models.load_model("NanoNet/NanoNet", compile=False)
model = tf.keras.models.load_model("my_model", compile=False)

seq, aa = utils.get_seq_aa("6zrv.pdb", "H")
one_hot = utils.generate_input("6zrv.pdb")

predicted_xyz = model.predict(np.expand_dims(one_hot, axis=0))[0]
utils.matrix_to_pdb(seq, predicted_xyz, "6zrv_network")

with open("6zrv_network.pdb") as ifile:
    predicted = "".join([x for x in ifile])
with open("6zrv.pdb") as ifile:
    true = "".join([x for x in ifile])

r,g,b = 255,0,0
print(f"\033[38;2;{r};{g};{b}mPredicted model \033[38;2;255;255;255m")
r,g,b = 0,0,255
print(f"\033[38;2;{r};{g};{b}mNative structure \033[38;2;255;255;255m")
view = py3Dmol.view(width=500, height=500)
view.addModelsAsFrames(predicted)
view.setStyle({'model': 0}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.addModelsAsFrames(true)
view.setStyle({'model': 1}, {"cartoon": {'arrows':True, 'color': 'blue'}})
view.zoomTo()
view.show()


### Function for reconstructing side chains and relaxing the model (using modeller)

In [ ]:
!mod10.1 modeller_side_chains.py 6zrv_network.pdb $seq
!mv pdb_seq.B99990001.pdb 6zrv_network_side_chains.pdb
!python align.py 6zrv.pdb 6zrv_network_side_chains.pdb

### View relaxed structure with side chains

In [ ]:
with open("6zrv_network_side_chains_aligned.pdb") as ifile:
    predicted = "".join([x for x in ifile])
with open("6zrv.pdb") as ifile:
    true = "".join([x for x in ifile])


r,g,b = 255,0,0
print(f"\033[38;2;{r};{g};{b}mPredicted model \033[38;2;255;255;255m")
r,g,b = 0,0,255
print(f"\033[38;2;{r};{g};{b}mNative structure \033[38;2;255;255;255m")
view = py3Dmol.view(width=500, height=500)
view.addModelsAsFrames(predicted)
view.setStyle({'model': 0}, {"cartoon": {'arrows':True, 'color': 'red'}, 'stick':{'color': 'red'}})
view.addModelsAsFrames(true)
view.setStyle({'model': 1}, {"cartoon": {'arrows':True, 'color': 'blue'}, 'stick':{'color': 'blue'}})
view.zoomTo()
view.show()
